**MLP classifier trained on the Fashion MNIST dataset and tuned using KerasTuner**

# Image classification using the Fashion MNIST dataset
We will train and evaluate an MLP on the Fashion MNIST dataset. It consists of 70.000 grayscale images of 28x28 pixels each, and there are 10 classes.
We will use KerasTuner for hyperparameter Tuning.

## Setup

In [1]:
# Common imports
import sys
import os
import tensorflow as tf
from tensorflow import keras

# to make this notebook's output stable across runs
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib.pyplot as plt

## Load the data

In [2]:
data = tf.keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = data.load_data()

In [3]:
# Show the size and dimension of the dataset.
X_train_full.shape

(60000, 28, 28)

In [4]:
# Each pixel intensity is represented as a byte (0 to 255).
X_train_full.dtype

dtype('uint8')

In [5]:
# Split the full training set into a validation set and a (smaller) training set,
# and scale the pixel intensities down to the 0-1 range and convert them to floats, by dividing by 255.
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

## Hyperparameter tuning with KerasTuner

In [6]:
# Install KerasTuner if you are using Google Colab (if you are running on your own computer
# it may also be necessary to install KerasTuner once from the command prompt).
if "google.colab" in sys.modules:
    %pip install -q -U keras_tuner

In [7]:
# Create a function that builds, compiles and returns a Keras model.
# Note that the function takes a HyperParameters object (hp) as a parameter, which it can
# use to define hyperparameters along with their range of possible values.
import keras_tuner as kt

def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=1, max_value=10, default=2)
    n_neurons = hp.Int("n_neurons", min_value=10, max_value=200)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model

In [8]:
# Use KerasTuner RandomSearch to tune hyperparameters
random_search_tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=60, overwrite=True,
    directory="my_mnist", project_name="my_rnd_search", seed=42)
random_search_tuner.search(X_train, y_train, epochs=40,
                           validation_data=(X_valid, y_valid))

Trial 49 Complete [00h 00m 44s]
val_accuracy: 0.8895999789237976

Best val_accuracy So Far: 0.8992000222206116
Total elapsed time: 00h 47m 14s

Search: Running Trial #50

Value             |Best Value So Far |Hyperparameter
5                 |10                |n_hidden
24                |190               |n_neurons
0.0032988         |0.0063534         |learning_rate

Epoch 1/40
1719/1719 [==============================] - 1s 559us/step - loss: 1.5745 - accuracy: 0.4446 - val_loss: 0.9582 - val_accuracy: 0.6278
Epoch 2/40
1719/1719 [==============================] - 1s 491us/step - loss: 0.8188 - accuracy: 0.6775 - val_loss: 0.7220 - val_accuracy: 0.7318
Epoch 3/40
1719/1719 [==============================] - 1s 466us/step - loss: 0.6891 - accuracy: 0.7395 - val_loss: 0.8133 - val_accuracy: 0.6808
Epoch 4/40
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6267 - accuracy: 0.7675 - val_loss: 0.5908 - val_accuracy: 0.7774
Epoch 5/40
1719/1719 [========================

In [9]:
# Display a summary of the best model:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

In [10]:
# Display the best model's accuracy measured on the validation set:
best_trial.metrics.get_last_value("val_accuracy")

In [11]:
# Evaluate the model's accuracy on the test set:
best_model = random_search_tuner.get_best_models(num_models=1)[0]
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)